In [ ]:
# ENVIRONMENT

from aisurveywriter.core.llm_handler import LLMHandler
import aisurveywriter.core.file_handler as fh
from aisurveywriter.utils import get_all_files_from_paths
from aisurveywriter.core.pipeline import PaperPipeline
from aisurveywriter.core.paper import PaperData
import aisurveywriter.tasks as tks

import os
os.environ["GOOGLE_API_KEY"]=fh.read_credentials("../credentials.yaml")["google_key"]

# llm = LLMHandler(model="qwen2.5:14b", model_type="ollama", temperature=0.5)
prompts = fh.read_yaml("../templates/prompt_config.yaml")
review = fh.read_yaml("../templates/review_config.yaml")

In [ ]:
# Serialize default prompt store

from aisurveywriter.store.prompt_store import PromptStore, default_prompt_store
import json

old = default_prompt_store()

with open("prompts-24022025.json", "w", encoding="utf-8") as f:
    json.dump(old.model_dump(), f, indent=2)

In [ ]:
# Manual RAG retrieval

from aisurveywriter.core.agent_rags import AgentRAG, RAGType
from aisurveywriter.core.text_embedding import EmbeddingsHandler

embed = EmbeddingsHandler("Snowflake/snowflake-arctic-embed-l-v2.0", "huggingface")
rag = AgentRAG(embed, bib_faiss_path="../out/refextract-bibdb.faiss", figures_faiss_path="../out/figures-rag.faiss", content_faiss_path="../out/content-rag.faiss",
               request_cooldown_sec=6)

query = r"Schematic representation of a Langmuir trough that contains the Wilhelmy plate for measuring surface pressure using an electrobalance and a surface potential probe.   Also shown is the dipper employed for transferring Langmuir/Blodgett onto a solid substrate"
rag.retrieve(RAGType.ImageData, query)

In [ ]:
# image caption extraction test
from aisurveywriter.core.pdf_processor import PDFProcessor

pdf = PDFProcessor(["../refexamples/filter-artigos-rafael/103.pdf"])
images = pdf.extract_images("test")


In [ ]:
print("\n".join([f"{i}: {image.caption}" for i,image in enumerate(images)]))

In [ ]:
text = pdf.pdf_documents[0][9].page_content
# text = text[text.find("9."):]
print(text)